# Harassment and Newcomer Retention

In this notebook we investigate how receiving harassment correlates with newcomer activity and retention. For the purposes of this study, our measures of harassment are classifiers over individual discussion comments for personal attacks, aggression and toxicity. These classifiers were developed [in previous work](https://arxiv.org/abs/1610.08914). We will investigate the relationship between harassment and newcomer retention through running regression models that use a measures of editing activity and harassment in time span t1 as independent variables and a measure of harassment in time span t2 as the dependent variable.

In [1]:
% matplotlib inline
import pandas as pd
from dateutil.relativedelta import relativedelta
import statsmodels.formula.api as sm
import requests

### Load Data for 2013, 2014, 2015 newcomers

The data used in this analysis includes:
1. all user and article talk page comments, labeled by harassment classifiers, except those generated by bots or templates
2. all newly registered users, who made at least one edit
3. edits per day per namespace for all newcomers
4. user warnings received by 2015 newcomers in
5. genders of all editors if available

In [2]:
# load scord diffs for 2015, drop admin and bot messages
usecols = [3,5,7,8,9,10,11,12,13]

nss = ['user', 'article']
years = [2011, 2012, 2013, 2014, 2015]

dfs = []

for year in years:
    for ns in nss:

        df = pd.read_csv("../../data/figshare/scored/comments_%s_%d.tsv.gz" % (ns, year),
                         sep = "\t",
                         compression = "gzip",
                         usecols = usecols)
        df['ns'] = ns
        df = df.query("bot == 0 and admin == 0")
        dfs.append(df)

df_annotated = pd.concat(dfs)
df_annotated['timestamp'] = pd.to_datetime(df_annotated['timestamp'])

In [3]:
df_annotated.head()

,timestamp,page_title,user_text,bot,admin,key,pred_attack_score,pred_aggression_score,pred_toxicity_score,ns
1,2011-12-08 10:44:41,27.114.174.55,49.213.35.189,0.0,0.0,0,0.108899,0.403929,0.061511,user
6,2011-12-08 14:32:18,Angelstar~enwiki,Kelly,0.0,0.0,0,0.001226,0.004074,0.000133,user
7,2011-12-08 15:25:12,Mxrider21,NawlinWiki,0.0,0.0,0,0.003002,0.002622,0.004874,user
19,2011-12-08 20:38:11,72.152.65.231,Vyselink,0.0,0.0,0,0.002464,0.003260,0.008621,user
20,2011-12-08 18:56:42,Imperious2780,Gaijin42,0.0,0.0,0,0.001708,0.005992,0.001267,user


In [4]:
# registration times of all editors who registered in 2015 and who made and edit
df_user_start = pd.read_csv("../../data/retention/newcomer_user_start.tsv", "\t")
df_user_start['registration_day'] = pd.to_datetime(df_user_start['registration_day'], format = '%Y%m%d')
df_user_start['first_edit_day'] = pd.to_datetime(df_user_start['first_edit_day'], format = '%Y%m%d')

In [5]:
# load edits per day in 2015 for new users in 2015
df_edits = pd.read_csv("../../data/retention/newcomer_daily_revision_counts.tsv", "\t")
df_edits['timestamp'] = pd.to_datetime(df_edits['day'].apply(lambda x: str(x)))

In [6]:
# load user warnings recieved by 2015_newcomers
df_uw = pd.read_csv("../../data/retention/newcomer_user_warnings.tsv", "\t")
df_uw['timestamp'] = pd.to_datetime(df_uw['rev_timestamp'])

In [7]:
# genders for all editors
df_gender = pd.read_csv("../../data/misc/genders.tsv", "\t")

In [ ]:
# create df of consolidated user level features
df_user = df_edits.drop_duplicates(subset = 'user_text')[['user_text']]
df_user = df_user.merge(df_gender, on = 'user_text', how = "left")[['user_text', 'gender']]
df_user['gender'] = df_user['gender'].fillna('unknown')
df_user = df_user.merge(df_user_start, on = 'user_text', how = "inner")[['user_text', 'gender', 'registration_day', 'first_edit_day']]
del df_user_start
del df_gender

### Group data by user
To be able help with extracting user level features, we group data sources above by user and store the results in a dedicated `User` object.

In [ ]:
# map data frames into dictionaries keyed by user
def gb_to_dict(gb):
    return { i:k for i,k in gb}

df_annotated_user_text_groups = gb_to_dict(df_annotated.groupby("user_text"))
df_annotated_page_title_groups =  gb_to_dict(df_annotated.query("ns == 'user'").groupby("page_title"))
df_edits_groups =  gb_to_dict(df_edits.groupby("user_text"))
df_user_groups =  gb_to_dict(df_user.groupby("user_text"))
df_uw_groups =  gb_to_dict(df_uw.groupby("page_title")) # page title is the recipient of the uw

In [ ]:
# collect User objects 
class User():
    def __init__(self, user_text, df_annotated_user_text_groups, df_annotated_page_title_groups, df_edits_groups, df_user_groups, df_uw_groups):
        self.user_text = user_text
        self.df_activity =  df_edits_groups.get(user_text, None)
        self.df_comments_made =  df_annotated_user_text_groups.get(user_text, None)
        self.df_comments_received = df_annotated_page_title_groups.get(user_text, None)
        self.df_uw = df_uw_groups.get(user_text, None)
        if self.df_comments_received is not None:
            self.df_comments_received = self.df_comments_received.query("ns == 'user' and user_text != page_title")
        self.gender = df_user_groups[user_text]['gender'].iloc[0]
        self.registration_day = df_user_groups[user_text]['registration_day'].iloc[0]
        self.first_edit_day = df_user_groups[user_text]['first_edit_day'].iloc[0]

### Select newcomer sample

We will select all newcomers who received some form harassment as determined by one of our comment-level harassment classifiers and a sample of 50000 randomly selected newcomers. This is to speed up implementation, but we can run the regressions on all data at the end.

In [ ]:
threshold = 0.425

new_user_texts = df_user.query("registration_day <= '2015-10-01'")[['user_text']]
print("Total New Users: ", new_user_texts.shape[0])

df_bad_comments = df_annotated.query("pred_attack_score > %f \
                                      or pred_aggression_score > %f \
                                      or pred_toxicity_score > %f" % (threshold, threshold, threshold))

attacked_users = df_bad_comments.query("ns == 'user' and user_text != page_title")\
                                .drop_duplicates(['page_title'])
    
attacked_users = attacked_users[['page_title']]
new_attacked_users = attacked_users.merge(new_user_texts, right_on = 'user_text', left_on = 'page_title')[['user_text']]
print("Total New Harrassed Users: ", new_attacked_users.shape[0])

# stratified sample
n_random = 100000
random_new_user_texts_sample = new_user_texts.sample(n_random)
new_user_texts_sample = pd.concat([random_new_user_texts_sample, new_attacked_users]).drop_duplicates()

#new_user_texts_sample = new_user_texts

user_objects = [User( user_text,
                      df_annotated_user_text_groups,
                      df_annotated_page_title_groups,
                      df_edits_groups,
                      df_user_groups, 
                      df_uw_groups) 
                for user_text in new_user_texts_sample['user_text']]

### Feature extraction

Our measures of user activity over a time span include:
1. number of edits in all namespaces
2. number of days active (a user is active on a day if they make at least on edit in any namespace)
3. number of edit sessions (an edit session is a sequence of edits without a gap of 60 minutes or more)
4. indicator of whether the user made at least one edit in any namespace


Our measures of harassment received/made over a time span are:
1. number of a comments received/made that classifier `clf` scored above `threshold`
2. number of a comments received/made that scored above `threshold` for any of our 3 harassment classifers
4. indicator of whether the user received/made at least one comment that scored above `threshold` for any of our 3 harassment classifiers


We also gather:
1. each users gender
2. and the number of user warnings the editor received

As mentioned above we, gather activity and harassment features for newcomers in timespan t1 and see how they correlate with activity features in timespan t2.

In the following analysis, the two time spans we are interested in are the first and second month after user registration.

In [ ]:
def select_month_since_registration(user,  activity, t):
    start = user.registration_day + relativedelta(months=(t-1))
    stop = user.registration_day + relativedelta(months= t)
    activity = activity[activity['timestamp'] < stop]
    activity = activity[activity['timestamp'] >= start]
    return activity

def count_edits(user, t):
    activity = user.df_activity
    activity = select_month_since_registration(user,  activity, t)
    return activity['n_revisions'].sum()

def count_days_active(user, t):
    activity = user.df_activity
    activity = select_month_since_registration(user,  activity, t)
    return len(activity.timestamp.unique())

def count_score_received_above_threshold(user, score, threshold, t):
    if user.df_comments_received is None:
        return 0
    
    comments = user.df_comments_received
    comments = select_month_since_registration(user,  comments, t)
    return (comments[score] > threshold).sum()

def count_score_made_above_threshold(user, score, threshold, t):
    if user.df_comments_made is None:
        return 0
    
    comments = user.df_comments_made
    comments = select_month_since_registration(user,  comments, t)
    return (comments[score] > threshold).sum()

def is_female(u):
    return int(u.gender == 'female')

def is_male(u):
    return int(u.gender == 'male')

def count_warnings_received(user, t):
    warnings = user.df_uw
    if warnings is None:
        return 0
    warnings = select_month_since_registration(user, warnings, t)
    return len(warnings)

def count_fraction_of_ns0_revisions_x(user, x, t):
    
    if user.df_activity is None:
        return 0
    
    activity = user.df_activity.query("ns=='0'")
    activity = select_month_since_registration(user,  activity, t)
        
    if activity['n_revisions'].sum() < 1:
        return 0
    
    return  float(activity[x].sum()) / activity['n_revisions'].sum()
    


In [ ]:
df_features = pd.DataFrame({
        'user_text' : [u.user_text for u in user_objects],
        'is_female' : [is_female(u) for u in user_objects],
        'is_male' : [is_male(u) for u in user_objects],
        't1_num_edits' : [count_edits(u, 1) for u in user_objects],
        't2_num_edits' : [count_edits(u, 2) for u in user_objects],
        't1_num_days_active' : [count_days_active(u, 1) for u in user_objects],
        't2_num_days_active' : [count_days_active(u, 2) for u in user_objects],
        't1_num_attacks_received' : [count_score_received_above_threshold(u, 'pred_attack_score',  threshold, 1) for u in user_objects],
        't1_num_aggression_received' : [count_score_received_above_threshold(u,  'pred_aggression_score',  threshold, 1) for u in user_objects],
        't1_num_toxicity_received' : [count_score_received_above_threshold(u,  'pred_toxicity_score',  threshold, 1) for u in user_objects],
        't1_num_attacks_made' : [count_score_made_above_threshold(u, 'pred_attack_score',  threshold, 1) for u in user_objects],
        't1_num_aggresssion_made': [count_score_made_above_threshold(u,  'pred_aggression_score',  threshold, 1) for u in user_objects],
        't1_num_toxicity_made': [count_score_made_above_threshold(u,  'pred_toxicity_score',  threshold, 1) for u in user_objects],
        't1_num_warnings_recieved' : [count_warnings_received(u, 1) for u in user_objects],
        't1_fraction_ns0_deleted' : [count_fraction_of_ns0_revisions_x(u, 'n_deleted_revisions', 1) for u in user_objects],
        't1_fraction_ns0_reverted' : [count_fraction_of_ns0_revisions_x(u, 'n_identity_reverted_revisions', 1) for u in user_objects],
        't1_fraction_ns0_productive' : [count_fraction_of_ns0_revisions_x(u, 'n_productive_revisions', 1) for u in user_objects],

    })

df_features.shape

df_features['t1_active'] = (df_features['t1_num_edits'] > 0).apply(int)
df_features['t2_active'] = (df_features['t2_num_edits'] > 0).apply(int)
df_features['t1_harassment_received'] = ((df_features['t1_num_attacks_received'] > 0) | (df_features['t1_num_aggression_received'] > 0) | (df_features['t1_num_toxicity_received'] > 0)).apply(int)
df_features['t1_harassment_made'] = ((df_features['t1_num_attacks_made'] > 0) | (df_features['t1_num_aggresssion_made'] > 0) | (df_features['t1_num_toxicity_made'] > 0)).apply(int)
df_features['has_gender'] = ((df_features["is_female"] == 1) | (df_features["is_male"] == 1)).apply(int)




df_features.index = df_features.user_text
del df_features['user_text']
df_features.shape

### Subset the full newcomers population

We are interested in running our regression models on the following subsets of the entire newcomer population:

1. User who made at least one edit in t1 (active)

In [ ]:
df_active = df_features.query('t1_active ==1')
print(df_active.shape[0])

### Regression Modeling

In this section we explore various regression models that use a measures of editing activity and harassment in time span t1 as independent variables and a measure of harassment in time span t2 as the dependent variable.

In [ ]:
df_reg = df_active

In [ ]:
def regress(df, formula, family = 'linear'):
    if family == 'linear':
        result = sm.ols(formula=f, data=df).fit()
    elif family == 'logistic':
        result = sm.logit(formula=f, data=df).fit(disp=0)
    else:
        print("Wrong Family")
    return result.summary().tables[1]

#### Logistic Regression: Does receiving harassment in t1 make you less likely to make an edit t2?

In [ ]:
f = "t2_active ~ t1_harassment_received"
regress(df_reg, f, family = 'logistic')

This model suggests that users who receive harassment have increased probability of being active in t2, compared to users who did not receive harassment. Lets control for how active the user was in t1 to see if the result holds.

In [ ]:
f ="t2_active ~ t1_harassment_received + t1_num_days_active"
regress(df_reg, f, family = 'logistic')

After controlling for the number of days a user was active in t1, we see that users receiving harassment have a decreased probability of activity in t2.

#### Linear Regression: Is harassment correlated with reduction in activity from t1 to t2?

Instead of running a logistic regression using an indicator for activity in t2 as our dependent variable, we will run a linear regression using the the number of days active in t2 as our dependent variable.

In [ ]:
f ="t2_num_days_active ~ t1_harassment_received"
regress(df_reg, f)

We see a similar results as above. Without controlling for activity in t1, users who receive harassment have, on average, more active days in t2.

In [ ]:
f= "t2_num_days_active ~ t1_num_days_active + t1_harassment_received"
regress(df_reg, f)

However, when we control for the number of days a user is active in t1, we see that users who receive harassment have fewer active days in t2. The coefficient is significantly less than 0 and larger in magnitude as on the number of active days in t1. Instead of using an indicator for whether the user received harassment, lets use the count of various types of harassment received (i.e personal attacks, aggression, toxicity)

In [ ]:
f= "t2_num_days_active ~ t1_num_days_active + t1_num_attacks_received"
regress(df_reg, f)

In [ ]:
f ="t2_num_days_active ~ t1_num_days_active + t1_num_aggression_received"
regress(df_reg, f)

In [ ]:
f="t2_num_days_active ~ t1_num_days_active + t1_num_toxicity_received"
regress(df_reg, f)

We see the same general pattern as above, accept that toxic comments received seem to have a weaker association with lower activity in t2 than personal attacks and aggression. Also, the magnitude of the coefficients decreased since we are using a count and not an indicator as above.

Before we regress an activity measure in t2 on an activity measure in t1 and multiple measures of harassment, lets see how our different measures of harassment correlate:

In [ ]:
from scipy.stats import pearsonr 
print(pearsonr(df_reg['t1_num_attacks_received'] ,  df_reg['t1_num_aggression_received']))
print(pearsonr(df_reg['t1_num_toxicity_received'] , df_reg['t1_num_aggression_received']))
print(pearsonr(df_reg['t1_num_toxicity_received'] , df_reg['t1_num_attacks_received']))

Personal attacks and aggression are very highly correlated. This is probably because both questions appeared on the same form. The toxicity measure has a lower though still high correlation with both personal attacks and aggression. Let's try a regression using toxicity and one of the two other measures:

In [ ]:
f="t2_num_days_active ~ t1_num_days_active + t1_num_toxicity_received + t1_num_attacks_received"
regress(df_reg, f)

This result is harder to interpret due to the strong correlation between `t1_num_toxicity_received` and `t1_num_attacks_received`.

#### Linear Regression: How do gender, harassment and activity interact?

Gender in Wikiedia is not well defined. After registering, users have the ability to report their gender in their user preferences. The vast majority of users do not report their gender. This may be because reporting their gender is not important to them, they don't want to report a gender, or they simply are unaware of the feature. There is anectodital evidence that users often report an incorrect gender. Overall, this means that we should expect users who report their gender to be different than the rest and we cannot be sure if reported genders are correct. Another caveat for the following analysis is that we do not know when the user reported their gender; they may have changed their user preference after our 2 month interval of interest.




In [ ]:
f="t1_harassment_received ~ has_gender"
regress(df_reg, f, family = 'logistic')

Users who supply a gender are more likely to receive harassment! Lets see if this is different for males and females:

In [ ]:
f="t1_harassment_received ~ is_female"
regress(df_reg.query("has_gender == 1"), f, family = 'logistic')

Females have an increased probability of receiving harassment in t1.

In [ ]:
f="t2_active ~ has_gender"
regress(df_reg, f)

Users who supply a gender are also more likely to be active in t2! Again, lets see if this is different for males and females:

In [ ]:
f="t2_active ~ is_female"
regress(df_reg.query("has_gender == 1"), f, family = 'logistic')

Females have a descreased probability of being active in t1, although the effect is not significant. Lets see what happens when we control for activity in t1.

In [ ]:
f="t2_num_days_active ~ t1_num_days_active + has_gender"
regress(df_reg, f)

Users who supply a gender appear to be more active in t2 even after controlling for activity in t1.

In [ ]:
f="t2_num_days_active ~ t1_num_days_active + is_female"
regress(df_reg.query("has_gender == 1"), f)

Females appear to have decreased activity in t2 even after controlling for activity in t1 compared to males.

In [ ]:
f="t2_num_days_active ~ t1_num_days_active + t1_harassment_received * has_gender"
regress(df_reg, f)

It seems like users who supply a gender and receive harassment have even more strongly reduced activity in t2 compared to users who do not supply a gender and get harassed.

In [ ]:
f="t2_num_days_active ~ t1_num_days_active + t1_harassment_received * is_female"
regress(df_reg.query("has_gender == 1"), f)

Although the effect is not significant, it seems like females who receive harassment have even more strongly reduced activity in t2.

#### Linear regression: addressing the bad newcomer confound

A serious potential confound in our analyses could be that the users who receive harassment are just bad faith newcomers or sock-puppets. They get attacked for their misbehavior and reduce their activity in t2 because they get blocked or because they never intended to stick around past their own attacks. To reduce this confound, we control for whether the user harassed anyone in t1 and for whether they received an user warning of any type:

In [ ]:
f="t2_num_days_active ~ t1_num_days_active + t1_harassment_received + t1_harassment_made * t1_harassment_received + t1_num_warnings_recieved * t1_harassment_received "
regress(df_active, f)

Even users who receive harassment but did not harass anyone or receive a user warning show reduced activity in t2.

#### Linear regression: addressing the bad newcomer confound

### Investigate some newcomer experiences

Our regression analyses have established that newcomer who receive harassment show a greater subsequent decline in activity than normal. Let's look at a few example of newcomers, what edits they made and how other interacted with them.